# S21 vs Freq sweep, continuous

In [4]:
# run once
#matplotlib
%gui qt

In [5]:
# import modules
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import json
import visa
import myplots, myutils # custom modules
import myplots2
import os.path
import time

# instrument drivers
import SIM
import AgilentN5230A
import LSCI332S
#import HittiteHMC
#import AgilentMXG

# instrumentinitialization
pna = AgilentN5230A.AgilentN5230A(16)
sim_glb = SIM.SIM(18, channel=3)
sim_chip = SIM.SIM(18, channel=1)
lsci = LSCI332S.LSCI332S(2)
#spec = HittiteHMC.HittiteHMC(18)
#spec = AgilentMXG.AgilentMXG(30)

In [22]:
rm=visa.ResourceManager()
rm.list_resources()


(u'visa://128.230.52.202/USB0::0x0957::0x1F01::MY47420076::INSTR',
 u'visa://128.230.52.202/ASRL1::INSTR',
 u'visa://128.230.52.202/ASRL5::INSTR',
 u'visa://128.230.52.202/ASRL6::INSTR',
 u'visa://128.230.52.202/ASRL8::INSTR',
 u'visa://128.230.52.202/ASRL10::INSTR',
 u'visa://128.230.52.202/GPIB0::19::INSTR',
 u'ASRL1::INSTR',
 u'ASRL10::INSTR',
 u'GPIB0::1::INSTR',
 u'GPIB0::2::INSTR',
 u'GPIB0::3::INSTR')

In [3]:
sim_chip.setVoltage(0)

In [21]:
####################################################################################
# set up parameter
startFreq, endFreq = 6.902e9, 6.9035e9
power = -50
numPoints, IFbandwidth, avg = 401, 100, 1
#volt = 0 # -0.65 # -2.05  #2.4
####################################################################################

# plot setup
plotobj= myplots2.plot1D(ylabel = 'S21', yunit='dB', xlabel='Frequency', xunit='Hz', title = 'S21 vs Freq'
                        , plotStyle='b-')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')

pna.setFreqStartEnd(startFreq, endFreq)
pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

#spec.pwrOff()
#sim_glb.setVoltage(volt)

# data initialization
S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])

################ sweep start ################

for i in range(1):
    # configure
    pna.avgClear()
        
    # fetch data from PNA
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    
    # plot update
    plotobj.update(freq/1e9, S21_mag_log)
    
     
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
print('Measurement Done!!')


Initialized S21 Measurement 'S21Meas'
Measurement Done!!


# S21 vs Freq vs Power, 2D

In [14]:
%%time
#########################################################################################
# set up parameters
startFreq, endFreq = 6.814e9, 6.81595e9 #6.775e9 , 6.945e9 #6.881e9, 6.885e9 # 6.904e9, 6.912e9
numPoints, avg = 201, 1
IFStart, IFEnd =500, 500 # Hz
startPwr, endPwr, stepPwr = -50,-10, 0.5
volt_glb_initial = 0
volt_chip_initial = 0

#save_path = 'Z:\User\JJ\SFQ2\C6' # no backslash at the end.
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W4\B5\BlueFors\S21vsFreqvsP'
#save_path = 'C:\Users\Lab User\Desktop\Albany1222'
#surfix='Q3_Vglb=0.328V' # surfix to the file name
surfix='Q5_LSS'
commentStr=''  # comment to be placed in the log file.
############################################################################################

# IFbandwidth scales up linearly as power increases.
IF_arr = np.linspace(IFStart, IFEnd, np.arange(startPwr, endPwr + stepPwr, stepPwr).size) 

# dictionary for parameters to be saved in json file. Solely for log.
xID, xStart, xEnd, xStep = 'Power (dBm)', startPwr, endPwr, stepPwr
yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd)
             , ('numPoints',numPoints), ('IFbandwidth start (Hz)', IF_arr[0]),('IFbandwidth end (Hz)', IF_arr[-1])
             , ('avg',avg), ('volt_glb_initial (V)',volt_glb_initial),('volt_chip_initial (V)',volt_chip_initial), ('comment', commentStr)])

# file setup
timeStr = time.strftime("%Y%m%d_%H%M%S")
FAA = float(lsci.tempGet('B')) # get temperature
baseStr = 'S21vsFvsP_fr{:.9g}_{:.9g}_Pr{}_{}_{}_T{:.4g}_{}_{}'
baseFileName = baseStr.format(startFreq/1e9, endFreq/1e9, startPwr, endPwr, stepPwr,FAA,surfix, timeStr)
dataFileName_mag, dataFileName_phase  = (os.path.join(save_path, baseFileName + '_mag.dat'), 
                                         os.path.join(save_path, baseFileName + '_phase.dat'))
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f:
    json.dump(para,f,indent=4, sort_keys=True)

#with open(logFileName, 'wb') as f_log:
#    for k, v in para.items():  # write log file.
#        f_log.write('# {} : {} \n'.format(k,v))
   
# plot setup
plotobj= myplots2.plot2D(ylabel = 'Frequency(Hz)', xlabel='Power (dBm)', title = 'S21 vs Freq vs Power')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')
pna.setFreqStartEnd(startFreq, endFreq)
#pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
#pna.setIF(IFbandwidth)
pna.pwrOn()

sim_glb.setVoltage(volt_glb_initial)
sim_chip.setVoltage(volt_chip_initial)

# data initialization
S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])
i=0
################ sweep start ################
for pwr in np.arange(startPwr, endPwr + stepPwr, stepPwr): # traverse in [pmin, pmax].
    
    clear_output()
    print('startPwr={:.3g} dBm, endPwr={:.3g} dBm, current power={:.3g} dBm'.format(startPwr, endPwr, pwr))
    
    # configure
    pna.setIF(IF_arr[i]) 
    pna.setPwr(pwr)
    pna.avgClear()
    i += 1
    
    # fetch data from PNA
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    
    if S21_mag_log_2D.size == 0: # when S21_mag_2D is empty
        S21_mag_log_2D = S21_mag_log
        S21_mag_log_2D.shape = (S21_mag_log.size, 1)
        S21_phase_2D = S21_phase
        S21_phase_2D.shape = (S21_phase.size, 1)
    else:
        S21_mag_log_2D = np.column_stack((S21_mag_log_2D, S21_mag_log))
        S21_phase_2D = np.column_stack((S21_phase_2D, S21_phase))
     
    # plot update
    extent = [startPwr, pwr, startFreq, endFreq]
    plotobj.update(S21_mag_log_2D, extent)
    
    
    # save data in each iteration
    f_mag = open(dataFileName_mag, 'w') 
    f_phase = open(dataFileName_phase, 'w') 
    np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
    np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
    f_mag.close()
    f_phase.close()
    
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
print('Measurement complete!!!')


startPwr=-50 dBm, endPwr=-10 dBm, current power=-10 dBm
Measurement complete!!!
Wall time: 57.7 s


In [ ]:
sim_chip.setVoltage(0)

# S21 vs Freq vs Flux, 2D

In [12]:
%%time
###########################################################################
# set up parameters
startFreq, endFreq = 6.73e9, 7.035e9 # 6.935e9, 6.9450e9 #6.7949e9 , 6.9676e9 #6.83e9, 6.894e9#6.856e9, 7.015e9 #6.85e9, 7.05e9
power = -50
numPoints, IFbandwidth, avg = 2001, 1000, 1
startVolt, endVolt, stepVolt = -20, 20, 0.05 #-20, 20, 0.2 #-5, 5, 0.05 #  #-15, 15, 0.4#-20, 20, 0.5 #-1, 1.5, 0.04
sweepVoltSrc = 2 # 1: Vchip, 2: Vglb

Vglb_init = startVolt
Vchip_init = 0
Vglb_endV = Vglb_init  # end voltage for both global and onchip flux
Vchip_endV = Vchip_init


#save_path = 'Z:\User\JJ\SFQ2\C6'
#save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W3\B7\BlueFors\Jan2017\S21vsFreqvsV'
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W4\B5\BlueFors\S21vsFreqvsV'
#surfix = ''
surfix = 'Vchip={:.3g}'.format(Vchip_init)
commentStr =''
############################################################################

# dictionary for parameters to be saved in json file. Solely for log.
xID, xStart, xEnd, xStep = 'Volt (V)', startVolt, endVolt, stepVolt
yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd), ('Power (dBm)', power)
             , ('numPoints',numPoints), ('IFbandwidth (Hz)', IFbandwidth) 
             , ('avg',avg), ('comment', commentStr)])

# file setup
FAA = float(lsci.tempGet('B')) # get temperature
baseStr = 'S21vsFvsV_fr{:.9g}_{:.9g}_Pr{:.4g}_V{}_{}_{}_T{:.4g}_{}'
baseFileName = baseStr.format(startFreq/1e9, endFreq/1e9, power, startVolt, endVolt, stepVolt, FAA, surfix)
dataFileName_mag, dataFileName_phase  = (os.path.join(save_path, baseFileName + '_mag.dat'), 
                                         os.path.join(save_path, baseFileName + '_phase.dat'))
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f:
    json.dump(para,f,indent=4, sort_keys=True)

# plot setup
plotobj= myplots2.plot2D(ylabel = 'Frequency', yunit='Hz', xlabel='Volt', xunit='V', title = 'S21 vs Freq vs Volt')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')
pna.setFreqStartEnd(startFreq, endFreq)
pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

sim_glb.setVoltage(Vglb_init)
sim_chip.setVoltage(Vchip_init)

time.sleep(2)

# data initialization
S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])

################ sweep start ################
for volt in np.arange(startVolt, endVolt + stepVolt, stepVolt): # traverse in [pmin, pmax].
    clear_output()
    print('startV={:.3g} V, endV={:.3g} V, current volt={:.3g} V'.format(startVolt, endVolt, volt))

    # configure
    if sweepVoltSrc==1:
        sim_chip.setVoltage(volt)
    elif sweepVoltSrc==2:
        sim_glb.setVoltage(volt)
        
    time.sleep(0.1)
    pna.avgClear()
    
    # fetch data from PNA
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    
    if S21_mag_log_2D.size == 0: # when S21_mag_2D is empty
        S21_mag_log_2D = S21_mag_log
        S21_mag_log_2D.shape = (S21_mag_log.size, 1)
        S21_phase_2D = S21_phase
        S21_phase_2D.shape = (S21_phase.size, 1)
    else:
        S21_mag_log_2D = np.column_stack((S21_mag_log_2D, S21_mag_log))
        S21_phase_2D = np.column_stack((S21_phase_2D, S21_phase))
     
    # plot update
    extent = [startVolt, volt, startFreq, endFreq]
    plotobj.update(S21_mag_log_2D, extent)
        
    # save data in each iteration
#     f_mag = open(dataFileName_mag, 'w') 
#     f_phase = open(dataFileName_phase, 'w') 
#     np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
#     np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
#     f_mag.close()
#     f_phase.close()
    
############## sweep end #########################    
# save at the end
f_mag = open(dataFileName_mag, 'w') 
f_phase = open(dataFileName_phase, 'w') 
np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
f_mag.close()
f_phase.close()

# Post measurement   
del S21_mag_log_2D, S21_phase_2D
pna.pwrOff()
sim_glb.setVoltage(Vglb_endV)
sim_chip.setVoltage(Vchip_endV)
print('Measurement done!!!')

startV=-20 V, endV=20 V, current volt=20 V
Measurement done!!!
Wall time: 39min 20s


In [12]:
extent[2]

6760000000.0

## S21 vs Freq vs Flux, 3D

In [6]:
%%time
###########################################################################
# set up parameters
startFreq, endFreq = 6.77e9, 6.81e9 # 6.935e9, 6.9450e9 #6.7949e9 , 6.9676e9 #6.83e9, 6.894e9#6.856e9, 7.015e9 #6.85e9, 7.05e9
power = -60
numPoints, IFbandwidth, avg = 801, 1000, 1
startVolt, endVolt, stepVolt = -20, 20, 0.2 #-5, 5, 0.05 #  #-15, 15, 0.4#-20, 20, 0.5 #-1, 1.5, 0.04
endV = 0  # end voltage for both global and onchip flux
startVolt1, endVolt1, stepVolt1 =-5.3, 0.7, 0.05

slop, Vglb_offset = -0.0659, -0.5  # Vglb = slope * Vchip + Vglb_offset. Vglb_offset is the y-intercept in Vchip(x-axxis) vs Vglb (y) axis.

#save_path = 'Z:\User\JJ\SFQ2\C6'
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W3\B7\BlueFors\Jan2017\S21vsFreqvsV\AlongNormVchipAxis'
surfix = 'Q1_VchipAxis'
commentStr= 'slope={:.3g}, Vglb_offset={:.3g}'.format(slope, Vglb_offset)

############################################################################

# dictionary for parameters to be saved in json file. Solely for log.
xID, xStart, xEnd, xStep = 'Volt (V)', startVolt, endVolt, stepVolt
yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
zID, zStart, zEnd, zStep = 'Volt (V)', startVolt1, endVolt1, stepVolt1
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd)
             ,('zID', zID), ('zStart', zStart), ('zEnd', zEnd), ('Power (dBm)', power)
             , ('numPoints',numPoints), ('IFbandwidth (Hz)', IFbandwidth) 
             , ('avg',avg), ('comment', commentStr)])

# log file setup
FAA = float(lsci.tempGet('B')) # get temperature
baseStr = 'S21vsFvsV_fr{:.9g}_{:.9g}_Pr{:.4g}_V{}_{}_{}_T{:.4g}_{}'
baseFileName = baseStr.format(startFreq/1e9, endFreq/1e9, power, startVolt, endVolt, stepVolt, FAA, surfix)
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f:
    json.dump(para,f,indent=4, sort_keys=True)

# plot setup
#plotobj= myplots.plot2D(ylabel = 'Frequency(Hz)', xlabel='Volt (V)', title = 'S21 vs Freq vs Volt')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')
pna.setFreqStartEnd(startFreq, endFreq)
pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

sim_glb.setVoltage(slope * startVolt + Vglb_offset)
sim_chip.setVoltage(startVolt)

time.sleep(2)

################ sweep start ################
for Vglb_offset in np.arange(startVolt1, endVolt1+stepVolt1, stepVolt1):
    # data initialization
    S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])

    #sim_glb.setVoltage(volt_outer)
    
    for volt_inner in np.arange(startVolt, endVolt + stepVolt, stepVolt): # traverse in [pmin, pmax].
        clear_output()
        print('Vglb_offset={:.3g}'.format(Vglb_offset))
        print('startV={:.3g} V, endV={:.3g} V, current volt={:.3g} V'.format(startVolt, endVolt, volt_inner))

        # configure
        sim_chip.setVoltage(volt_inner)
        sim_glb.setVoltage(slope * volt_inner + Vglb_offset)
        time.sleep(0.1)
        pna.avgClear()

        # fetch data from PNA
        freq, S21 = pna.getData(cplx=True) # S21 in linear scale

        # S21 conversion
        S21_real, S21_imag = np.real(S21), np.imag(S21)
        S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
        S21_mag_log = 20.0 * np.log10(S21_mag)    # dB

        if S21_mag_log_2D.size == 0: # when S21_mag_2D is empty
            S21_mag_log_2D = S21_mag_log
            S21_mag_log_2D.shape = (S21_mag_log.size, 1)
            S21_phase_2D = S21_phase
            S21_phase_2D.shape = (S21_phase.size, 1)
        else:
            S21_mag_log_2D = np.column_stack((S21_mag_log_2D, S21_mag_log))
            S21_phase_2D = np.column_stack((S21_phase_2D, S21_phase))

        # plot update
        #extent = [startVolt, volt, startFreq, endFreq]
        #plotobj.update(S21_mag_log_2D, extent)
     
    sim_chip.setVoltage(endV)
    time.sleep(3)
     
    # save at the end of inner loop
    fileIdStr = '_Vglb_offset={:.3g}'.format(Vglb_offset)
    dataFileName_mag, dataFileName_phase  = (os.path.join(save_path, baseFileName + fileIdStr + '_mag.dat'), 
                                             os.path.join(save_path, baseFileName + fileIdStr + '_phase.dat'))
    f_mag = open(dataFileName_mag, 'w') 
    f_phase = open(dataFileName_phase, 'w') 
    np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
    np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
    f_mag.close()
    f_phase.close()
############## sweep end #########################    

# Post measurement   
del S21_mag_log_2D, S21_phase_2D
pna.pwrOff()
sim_glb.setVoltage(endV)
sim_chip.setVoltage(endV)
print('Measurement done!!!')

Vglb_offset=0.7
startV=-20 V, endV=20 V, current volt=20 V
Measurement done!!!
Wall time: 8h 42min 17s


# S21 vs flux1 vs flux2, 2D

In [14]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [13]:
import sys
for i in dir():
    try:
        print (i, eval(i).nbytes )
    except:
        print (i, sys.getsizeof(eval(i)) )

('AgilentN5230A', 48)
('FAA', 24)
('IFbandwidth', 24)
('In', 192)
('LSCI332S', 48)
('Out', 272)
('S21', 800)
('S21_imag', 400)
('S21_mag', 400)
('S21_mag_log', 400)
('S21_mag_log_2D', 1288008)
('S21_mag_log_avg', 8)
('S21_phase', 400)
('S21_phase_2D', 1288008)
('S21_phase_avg', 8)
('S21_real', 400)
('SIM', 48)
('Vglb_offset', 24)
('_', 33)
('__', 33)
('___', 33)
('__builtin__', 48)
('__builtins__', 48)
('__doc__', 97)
('__name__', 41)
('__package__', 16)
('_dh', 72)
('_i', 9784)
('_i1', 94)
('_i10', 8476)
('_i11', 9776)
('_i12', 9784)
('_i13', 300)
('_i2', 1124)
('_i3', 8380)
('_i4', 9716)
('_i5', 9722)
('_i6', 9724)
('_i7', 8414)
('_i8', 8444)
('_i9', 8450)
('_ih', 192)
('_ii', 9776)
('_iii', 8476)
('_oh', 272)
('_sh', 48)
('avg', 24)
('baseFileName', 103)
('baseStr', 97)
('clear_output', 112)
('comment', 62)
('commentStr', 64)
('data1_2D', 65448)
('data2_2D', 65448)
('dataFileName1', 188)
('dataFileName2', 190)
('dataFileName_mag', 206)
('dataFileName_phase', 208)
('endFreq', 24)
('e

In [3]:
%%time
##############################################################################
# set up parameters
readoutPwr = -60
readoutFreq = 6.7962e9 #6.7917e9 #6.792e9 #6.82e9 
numPoints, IFbandwidth, avg = 50, 500, 1
startVolt1, endVolt1, stepVolt1 =-5, 5, 0.5 #-10, 10, 0.2 #-10, 10, 0.05 #-1.2, 1, 0.025 # outer for-loop
startVolt2, endVolt2, stepVolt2 =-10,10, 0.5 #-20, 20, 0.5 #-20, 20, 0.5 # y axis. Inner for-loop.
endVolt = 0

surfix = 'Q1_V2=onchip'
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W3\B7\BlueFors\Jan2017_2nd\S21vsV1vsV2'
commentStr='with spec setup'
##############################################################################

xID, xStart, xEnd, xStep = 'Volt1 (V)', startVolt1, endVolt1, stepVolt1
yID, yStart, yEnd, yStep = 'Volt2 (V)', startVolt2, endVolt2, stepVolt2
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd)
             , ('readoutFreq (Hz)',readoutFreq),('power (dBm)',readoutPwr)
             , ('numPoints',numPoints), ('IFbandwidth (Hz)', IFbandwidth) 
             , ('avg',avg), ('comment', commentStr)])

# file setup
FAA = float(lsci.tempGet('B')) # get temperature
baseStr = 'S21vsV1vsV2_fr{:.9g}_Pr{:.4g}_V1_{}_{}_{}_V2_{}_{}_{}_T{:.4g}_{}'
baseFileName = baseStr.format(readoutFreq/1e9, readoutPwr, startVolt1, endVolt1, stepVolt1, startVolt2, endVolt2, stepVolt2, FAA, surfix)
dataFileName1  = os.path.join(save_path, baseFileName + '_mag.dat')
dataFileName2  = os.path.join(save_path, baseFileName + '_phase.dat')
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f_log:
    json.dump(para, f_log, indent=4, sort_keys=True)

# plot setup
plotobj= myplots.plot2D_4sub(xlabel1 = 'Voltage2 (V)', ylabel1='S21 (dB)', 
                             xlabel2 = 'Voltage2 (V)', ylabel2='Phase(degree)',
                             xlabel3 = 'Voltage, Vglb (V)', ylabel3='Voltage, Vchip (V)',
                             xlabel4 = 'Voltage, Vglb (V)', ylabel4='Voltage, Vchip (V)',
                             title = 'S21 vs Global Flux vs Gradiometric Flux', figsize=(12,8))

#plotobj = myplots2.plot2D(xlabel1='Voltage, Vglb', xunit1='V', ylabel = 'S21', yunit='dB', xlabel2='Voltage, V xlabel='Voltage, Vchip', xunit='V', title = 'S21 vs Vglb vs Vchip')

# preconfigure instrument
pna.setupMeas()
pna.setSweepType(mode='CW')
pna.setCWFreq(readoutFreq)
pna.setPwr(readoutPwr)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

sim_glb.setVoltage(startVolt1)
time.sleep(2)
sim_chip.setVoltage(startVolt2)
time.sleep(2)

# data initialization
data1_2D, data2_2D = np.array([]), np.array([])

################ sweep start ################
#for specFreq in arange(startFreq, endFreq + stepFreq, stepFreq): # traverse in [pmin, pmax].
for volt1 in np.arange(startVolt1, endVolt1 + stepVolt1, stepVolt1):
    #configure para
    sim_glb.setVoltage(volt1)
    
    # data initialization
    xdata1, ydata1, ydata2 = np.array([]), np.array([]), np.array([])
    
    for volt2 in np.arange(startVolt2, endVolt2 + stepVolt2, stepVolt2):
        clear_output()
        print('startVolt1={:.3g}, endtvolt1={:.3g}'.format(startVolt1, endVolt1))
        print('startVolt2={:.3g}, endtvolt2={:.3g}'.format(startVolt2, endVolt2))
        print('volt1={:.3g}, volt2={:.3g}'.format(volt1, volt2)) 
        
        # configure
        pna.avgClear()
        sim_chip.setVoltage(volt2)
    
        # fetch data from PNA
        freq, S21= pna.getData(cplx=True) # S21 in linear scale
    
        # S21 conversion
        S21_real, S21_imag = np.real(S21), np.imag(S21)
        S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
        S21_mag_log = 20.0 * np.log10(S21_mag)    # dB

        S21_mag_log_avg = np.average(S21_mag_log) 
        S21_phase_avg = np.average(S21_phase)

        xdata1 = np.append(xdata1, volt2)
        ydata1 = np.append(ydata1, S21_mag_log_avg)
        ydata2 = np.append(ydata2, S21_phase_avg)

        # plot and update
        plotobj.update1D(xdata1, ydata1, xdata1, ydata2)

    
    if data1_2D.size == 0: # when S21_mag_2D is empty
        data1_2D = ydata1 # make 2D array
        data1_2D.shape = (data1_2D.size, 1)
        data2_2D = ydata2
        data2_2D.shape = (data2_2D.size, 1)
        
    else:
        data1_2D = np.column_stack((data1_2D, ydata1))
        data2_2D = np.column_stack((data2_2D, ydata2))
             
    # plot and update
    extent = [startVolt1, volt1, startVolt2, endVolt2]
    plotobj.update2D(data1_2D, data2_2D, extent, cmap='Blues_r')
    
    # save data in each iteration
    f_mag = open(dataFileName1, 'w') 
    f_phase = open(dataFileName2, 'w') 
    np.savetxt(f_mag, data1_2D, fmt='%.9g', delimiter='\t') 
    np.savetxt(f_phase, data2_2D, fmt='%.9g', delimiter='\t')
    f_mag.close()
    f_phase.close()
     
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
pna.setSweepType(mode='linear')
sim_glb.setVoltage(endVolt)
sim_chip.setVoltage(endVolt)
f_log.close()
print('Measurement done!!!')

startVolt1=-5, endtvolt1=5
startVolt2=-10, endtvolt2=10
volt1=5, volt2=10
Measurement done!!!
Wall time: 10min 5s


# S21 vs Freq vs Flux (linear comb. of flux1 and flux2), 2D

In [104]:
reload(myplots)

<module 'myplots' from 'myplots.py'>

In [10]:
%%time
###########################################################################
# set up parameters
startFreq, endFreq =  6.77e9, 6.81e9 #6.83e9, 6.894e9#6.856e9, 7.015e9 #6.85e9, 7.05e9
power = -60
numPoints, IFbandwidth, avg = 801, 1000, 1
startVolt, endVolt, stepVolt =-20, 20, 0.2#-20, 20, 0.5 #-1, 1.5, 0.04
endV = 0

slope, Vglb_offset =-0.3378,2.4 # 0.2107, 0# 2.484#-0.3378, -0.9641# -0.0659, 0.7   # offset means Vglb when Vchip=0

#save_path = 'Z:\User\JJ\sput\M5_1_spec'
save_path = 'Z:\User\Jaseung\projects\CSFQ\measurement\W3\B7\BlueFors\Jan2017\S21vsFreqvsV'
surfix = 'Q1_xID=Vchip_slope{}_Vglb_offset{}'.format(slope, Vglb_offset)
comment= 'slope={:.3g}, Vglb_offset={:.3g}'.format(slope, Vglb_offset)

plottitle = 'S21 vs Freq vs Vchip, Vglb_offset={:.3g}'.format(Vglb_offset)

############################################################################

# dictionary for parameters to be saved in json file. Solely for log.
xID, xStart, xEnd, xStep = 'Volt (V)', startVolt, endVolt, stepVolt
yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)
para = dict([('xID', xID), ('xStart', xStart), ('xEnd', xEnd)
             , ('yID', yID), ('yStart', yStart), ('yEnd', yEnd), ('Power (dBm)', power)
             , ('numPoints',numPoints), ('IFbandwidth (Hz)', IFbandwidth) 
             , ('avg',avg), ('comment', comment)])

# file setup
FAA = float(lsci.tempGet('B')) # get temperature
baseStr = 'S21vsFvsV_fr{:.9g}_{:.9g}_Pr{:.4g}_V{}_{}_{}_T{:.4g}_{}'
baseFileName = baseStr.format(startFreq/1e9, endFreq/1e9, power, startVolt, endVolt, stepVolt, FAA, surfix)
dataFileName_mag, dataFileName_phase  = (os.path.join(save_path, baseFileName + '_mag.dat'), 
                                         os.path.join(save_path, baseFileName + '_phase.dat'))
logFileName =  os.path.join(save_path, baseFileName + '.json')
with open(logFileName, 'w') as f:
    json.dump(para,f,indent=4, sort_keys=True)

# plot setup
plotobj= myplots.plot2D(ylabel = 'Frequency(Hz)', xlabel='Volt (V)', title = plottitle)

# preconfigure instrument
pna.setupMeas()
pna.setSweepType('linear')
pna.setFreqStartEnd(startFreq, endFreq)
pna.setPwr(power)
pna.avgCount(avg)
pna.numPoints(numPoints)
pna.setIF(IFbandwidth)
pna.pwrOn()

sim_glb.setVoltage(slope*startVolt+Vglb_offset)
sim_chip.setVoltage(startVolt)
time.sleep(2)

# data initialization
S21_mag_log_2D, S21_phase_2D = np.array([]), np.array([])

################ sweep start ################
for volt_chip in np.arange(startVolt, endVolt + stepVolt, stepVolt): # traverse in [pmin, pmax].
    # configure
    volt_glb =  slope * volt_chip + Vglb_offset
    #volt_glb = -1/slope* (volt_chip-20) - 0.55
    clear_output()
    print('volt_glb={:.3g}, volt_chip={:.3g}'.format(volt_glb, volt_chip))
    sim_chip.setVoltage(volt_chip)
    sim_glb.setVoltage(volt_glb)
    
    time.sleep(0.1)
    pna.avgClear()
    
    # fetch data from PNA
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale
    
    # S21 conversion
    S21_real, S21_imag = np.real(S21), np.imag(S21)
    S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
    S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
    
    if S21_mag_log_2D.size == 0: # when S21_mag_2D is empty
        S21_mag_log_2D = S21_mag_log
        S21_mag_log_2D.shape = (S21_mag_log.size, 1)
        S21_phase_2D = S21_phase
        S21_phase_2D.shape = (S21_phase.size, 1)
    else:
        S21_mag_log_2D = np.column_stack((S21_mag_log_2D, S21_mag_log))
        S21_phase_2D = np.column_stack((S21_phase_2D, S21_phase))
     
    # plot update
    extent = [startVolt, volt_chip, startFreq, endFreq]
    plotobj.update(S21_mag_log_2D, extent)
        
    # save data in each iteration
    f_mag = open(dataFileName_mag, 'w') 
    f_phase = open(dataFileName_phase, 'w') 
    np.savetxt(f_mag, S21_mag_log_2D, fmt='%.9g', delimiter='\t') 
    np.savetxt(f_phase, S21_phase_2D, fmt='%.9g', delimiter='\t')
    f_mag.close()
    f_phase.close()
    
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
sim_glb.setVoltage(endV)
sim_chip.setVoltage(endV)
print('Measurement done!!!')

volt_glb=-4.36, volt_chip=20
Measurement done!!!
Wall time: 6min 33s
